# 第4回講義 宿題

## 課題. MNISTデータセットを多層パーセプトロン(MLP)で学習せよ

### 注意
- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください\
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください

- MLPの実装にTheanoなどのライブラリを使わないでください

### ヒント
- 出力yはone-of-k表現
- 最終層の活性化関数はソフトマックス関数, 誤差関数は多クラス交差エントロピー
- 最終層のデルタは教科書参照

次のコードが**事前**に実行されます


```python
from __future__ import division
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np

mnist = fetch_mldata('MNIST original')
mnist_X, mnist_y = shuffle(mnist.data, mnist.target.astype('int32'))

mnist_X = mnist_X / 255.0

train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=??) # random_stateはこちらで与えます
```

次のセルのhomework関数を完成させて提出してください
- パッケージのインポートなど、必要な物はすべて書いてください

In [ ]:
# from __future__ import division
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

import numpy as np

def load_mnist():
    mnist = fetch_mldata('MNIST original')
    mnist_X, mnist_y = shuffle(mnist.data, mnist.target.astype('int32'))

    mnist_X = mnist_X / 255.0

    train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=42)

    return (train_X, test_X, train_y, test_y)

def check_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    pred_y = homework(train_X, test_X, train_y)
    return f1_score(test_y, pred_y, average='macro')

if 'homework' in globals():
    result = check_homework()
    print "No Error Occured!"

In [1]:
def homework(train_X, test_X, train_y):
    np.seterr(divide='ignore', invalid='ignore', over = 'ignore')
    layer_num = 2
    out_dim = 10
    eps = 1.0
    dims = [len(train_X[0]),40,out_dim]

    w = [np.random.uniform(low = -0.08, high = 0.08, size = (dims[0],dims[1])).astype("float64")]
    b = [np.zeros(dims[1]).astype("float64")]
    u = [np.zeros(dims[1]).astype("float64")]
    z = [np.zeros(dims[1]).astype("float64")]
    delta = [np.zeros(dims[1]).astype("float64")]

    for i in range(1, layer_num):
        w.append(np.random.uniform(low = -0.08, high = 0.08, size = (dims[i],dims[i + 1])).astype("float64"))
        b.append(np.zeros(dims[i + 1]).astype("float64"))
        u.append(np.zeros(dims[i + 1]).astype("float64"))
        z.append(np.zeros(dims[i + 1]).astype("float64"))
        delta.append(np.zeros(dims[i + 1]).astype("float64"))

    for epoch in xrange(10):
        for x, y in zip(train_X, train_y):
            tmp_z = x[np.newaxis, :]
            y2 = np.zeros((1,10))
            y2[0,y] = 1

            #f_prop                                                                                                                         
            for i in xrange(layer_num - 1):
                if i == 0:
                    u[i] = np.dot(tmp_z, w[i]) + b[i]
                else:
                    u[i] = np.dot(z[i - 1], w[i]) + b[i]
                u[i][np.where(u < -20)] = -20
                z[i] = 1 / (1 + np.exp(-u[i])) #sigmoid 
            u[layer_num - 1] = np.dot(z[layer_num - 2], w[layer_num - 1]) + b[layer_num - 1]
            u[layer_num - 1][np.where(u < -20)] = -20
            z[layer_num - 1] = np.exp(u[layer_num -1]) / np.sum(np.exp(u[layer_num - 1]), axis = 1, keepdims = True) #softmax               
            #cost = np.sum(-y2 * np.log(z[layer_num - 1]) - (1 - y2)*np.log(1 - z[layer_num - 1]))
            tmp_delta = z[layer_num - 1] - y2

            #b_prop                                                                                                                         
            for i in xrange(layer_num):
                if i == 0:
                    delta[layer_num - 1] = tmp_delta
                else:
                    delta[layer_num - 1 - i] = np.dot(delta[layer_num - i], w[layer_num - i].T) * (1 / ( 1 + np.exp(u[layer_num - 1 - i]))) * (1 - (1 / ( 1 + np.exp(u[layer_num - 1 - i])))) #deriv_sigmoid
            #update parameter                                                                                                               
            for i in xrange(layer_num):
                if i == 0:
                    dW = np.dot(tmp_z.T, delta[i])
                    db = np.dot(np.ones(len(tmp_z)), delta[i])
                else:
                    dW = np.dot(z[i - 1].T, delta[i])
                    db = np.dot(np.ones(len(z[i -1])), delta[i])
                w[i] = w[i] - eps * dW
                b[i] = b[i] - eps * db
    #test                                                                                                                               
    pred_y = np.zeros(len(test_X)).astype("int32")
    for j in xrange(len(test_X)):
        tmp_z = test_X[j][np.newaxis, :]
        for i in xrange(layer_num):
            if i == 0:
                u[i] = np.dot(tmp_z, w[i]) + b[i]
            else:
                u[i] = np.dot(z[i - 1], w[i]) + b[i]
            z[i] = 1 / (1 + np.exp(-u[i])) #sigmoid                                                                                 
        u[layer_num - 1] = np.dot(z[layer_num - 2], w[layer_num - 1]) + b[layer_num - 1]
        z[layer_num - 1] = np.exp(u[layer_num -1]) / np.sum(np.exp(u[layer_num - 1]), axis = 1, keepdims = True) #softmax               
        pred_y[j] = np.argmax(z[layer_num - 1])

    return pred_y
